In [1]:
from elasticsearch import Elasticsearch 
import util as ut
from elasticsearch import helpers
from rdflib import Graph
import collections

In [2]:
# Connect to the elastic cluster
es=Elasticsearch([{'host':'localhost','port':9200}])

In [4]:
# get all indexes
#es.indices.get_alias("*")

In [65]:
#if es.indices.exists(index='rdf'):
#es.indices.delete(index='opal_distribution')
#es.indices.delete(index='opal_dataset')
#es.indices.delete(index='opal_measurements')

{'acknowledged': True}

In [66]:
es.indices.create(index='opal_dataset')
es.indices.create(index='opal_distribution')
es.indices.create(index='opal_measurements')

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'opal_measurements'}

In [67]:
g = Graph()
g.parse("KG/10000.nt", format="n3")
#g.parse("KG/10.nt", format="n3")

<Graph identifier=Nd9601ae976f84dbdb31cc9c33657d37f (<class 'rdflib.graph.Graph'>)>

In [68]:
'{0} number of triples'.format(len(g))

'100001 number of triples'

In [69]:
prefix_uri='http://projekt-opal.de/'
profiles=[prefix_uri+'dataset/',prefix_uri+'distribution/',prefix_uri+'measurement']

In [70]:
dataset_documents=dict()
distribution_documents=dict()
measurement_documents=dict()

for t in g:
    s,p,o=str(t[0]),str(t[1]),str(t[2])
    if profiles[0] in s:
        # Important
        dataset_documents.setdefault(s, {}).setdefault(p,[]).append(o)
    elif profiles[1] in s:
        # Important
        distribution_documents.setdefault(s, {}).setdefault(p,[]).append(o)
    elif profiles[2] in s:
        measurement_documents.setdefault(s, {}).setdefault(p,[]).append(o)
        

In [73]:
def gendata(index,X):
    for dataset_uri,body in X.items():
        yield {
            "_index": index,
            "_id":dataset_uri,
            "_type": "document",
            "_source": body
        }

In [74]:
helpers.bulk(es, gendata('opal_dataset',dataset_documents))

helpers.bulk(es, gendata('opal_distribution',distribution_documents))

helpers.bulk(es, gendata('opal_measurements',measurement_documents))

(1250, [])

### Indexing completed



In [88]:
# to get the number of documents
print(es.count(index='opal_dataset'))

print(es.count(index='opal_distribution'))

print(es.count(index='opal_measurements'))


{'count': 1250, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}
{'count': 2204, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}
{'count': 6268, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


In [86]:
res=es.get(index="opal_dataset", id='http://projekt-opal.de/dataset/https___europeandataportal_eu_set_data_dea0902b_a26f_4404_945f_9e8bd7def6c6_1')

In [103]:
def print_search_stats(results):
    print("=" * 80)
    print(
        "Total %d found in %dms" % (results["hits"]["total"]["value"], results["took"])
    )
    print("-" * 80)

### Example Queries 

{'count': 1250, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}
{'count': 2204, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}
{'count': 6268, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


In [17]:
res = es.get(index="opal_dataset", id='http://projekt-opal.de/dataset/https___europeandataportal_eu_set_data_ddt05_protection_du_patrimoine_architectural_et_urbain_generatrice_de_servitudes_ac4_des_hautes_alpes')

In [22]:
res['_source']

{'http://purl.org/dc/terms/publisher': ['f31e276dd8a16488892558a67a639adcfb15044',
  'f31e276dd8a16488892558a67a639adcfb15043'],
 'http://www.w3.org/ns/dqv#hasQualityMeasurement': ['http://projekt-opal.de/measurement2071402',
  'http://projekt-opal.de/measurement2071399',
  'http://projekt-opal.de/measurement2071403',
  'http://projekt-opal.de/measurement2071404',
  'http://projekt-opal.de/measurement2071400',
  'http://projekt-opal.de/measurement2071401',
  'http://projekt-opal.de/measurement2071405'],
 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': ['http://www.w3.org/ns/dcat#Dataset'],
 'http://www.w3.org/ns/dcat#distribution': ['http://projekt-opal.de/distribution/https___europeandataportal_eu_set_distribution_0e362a95_fb1f_4f21_8b10_d46ef5ae9503',
  'http://projekt-opal.de/distribution/https___europeandataportal_eu_set_distribution_19d1eb39_7aa3_42c2_8068_a684a2fe99bc',
  'http://projekt-opal.de/distribution/https___europeandataportal_eu_set_distribution_c65430f6_160d_456c_9f5

In [162]:
# Search for documents whose description contains framework
result=es.search(index='opal_dataset',
         body={
             "query":{
                 "bool":{
                     "must":{"match":{"http://purl.org/dc/terms/description":"@fr"}}
                 }
             }
         })

print_search_stats(result)

Total 70 found in 2ms
--------------------------------------------------------------------------------


In [161]:
list(result['hits']['hits'][0]['_source'].keys())

['http://www.w3.org/ns/dqv#hasQualityMeasurement',
 'http://purl.org/dc/terms/spatial',
 'http://www.w3.org/ns/dcat#keyword',
 'http://www.w3.org/ns/dcat#distribution',
 'http://purl.org/dc/terms/description',
 'http://www.w3.org/ns/dcat#theme',
 'http://purl.org/dc/terms/title',
 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type']

In [140]:
hits_docs=result['hits']['hits']

In [152]:
for i in hits_docs:
    
    index=i['_id']
    
    
  #  description=i['_source']['http://purl.org/dc/terms/description']
 #   print(index)
#    print(description)
    distribution=i['_source']['http://www.w3.org/ns/dcat#distribution'][0]
   
    resp = es.get(index='opal_distribution', id=distribution)
    print(resp)

    break

{'_index': 'opal_distribution', '_type': '_doc', '_id': 'http://projekt-opal.de/distribution/https___europeandataportal_eu_set_distribution_4f55bce7_405c_4ff7_83f7_1744c7bfb0b6', '_version': 1, '_seq_no': 1977, '_primary_term': 1, 'found': True, '_source': {'http://purl.org/dc/terms/description': [''], 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': ['http://www.w3.org/ns/dcat#Distribution'], 'http://purl.org/dc/terms/title': ['view'], 'http://www.w3.org/ns/dcat#accessURL': ['http://ext-geodata.lansstyrelsen.se/arcgis/services/WMS/LST_wms_miljodata/MapServer/WMSserver?']}}


In [163]:
# Search for documents whose description contains framework
result=es.search(index='opal_dataset',
         body={
             "query":{
                 "bool":{
                     "must":{"match":{"http://purl.org/dc/terms/description":" @fr"}}
                 }
             }
         })

print_search_stats(result)

Total 70 found in 2ms
--------------------------------------------------------------------------------


In [56]:

from elasticsearch import Elasticsearch,helpers

In [57]:
def gendata():
    mywords = ['foo', 'bar', 'baz']
    for word in mywords:
        yield {
            "_index": "mywords",
            "_type": "document",
            "doc": {"word": word},
        }

helpers.bulk(es, gendata())

(3, [])

In [93]:
es.count(index='mywords')

{'count': 3,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [95]:
es.get(index="bank", id=1)

{'_index': 'bank',
 '_type': '_doc',
 '_id': '1',
 '_version': 1,
 '_seq_no': 0,
 '_primary_term': 1,
 'found': True,
 '_source': {'account_number': 1,
  'balance': 39225,
  'firstname': 'Amber',
  'lastname': 'Duke',
  'age': 32,
  'gender': 'M',
  'address': '880 Holmes Lane',
  'employer': 'Pyrami',
  'email': 'amberduke@pyrami.com',
  'city': 'Brogan',
  'state': 'IL'}}

In [115]:
#To start with, this example groups all the accounts by state, and then returns the top 10 (default) states sorted by count descending (also default):
    
es.search(index='bank',
         body={
  "size": 0,
  "aggs": {
    "group_by_state": {
      "terms": {
        "field": "state.keyword"
      }
    }
  }
}
)


{'took': 35,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1000, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'group_by_state': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 743,
   'buckets': [{'key': 'TX', 'doc_count': 30},
    {'key': 'MD', 'doc_count': 28},
    {'key': 'ID', 'doc_count': 27},
    {'key': 'AL', 'doc_count': 25},
    {'key': 'ME', 'doc_count': 25},
    {'key': 'TN', 'doc_count': 25},
    {'key': 'WY', 'doc_count': 25},
    {'key': 'DC', 'doc_count': 24},
    {'key': 'MA', 'doc_count': 24},
    {'key': 'ND', 'doc_count': 24}]}}}